In [190]:
import pandas as pd
import numpy as np

In [191]:
def draw_sudoku(board):
    print("+" + "---+"*9)
    for i, row in enumerate(board):
        print(("|" + " {}   {}   {} |"*3).format(*[x if x != 0 else "0" for x in row]))
        if i % 3 == 2:
            print("+" + "---+"*9)

In [192]:
def read_csv(name):
    df = pd.read_csv(name, header=None)
    matrix = df.to_numpy()
    d=[]
    x = 0
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if(matrix[i,j] > 0):
                d.append( [[i,j],matrix[i,j]])
                x+=1
    print(fitness(matrix))
    draw_sudoku(matrix)
    return matrix, d

In [193]:
def initial_population(arr, n):    # generate n random matrix with values from 1->9
    x=y=val=-1
    population = []
    for i in range(n):
        population.append(np.zeros((9,9)))
    for j in range(n):
        mat = np.random.randint(low=1,high=10, size=(9, 9))
        #print(mat)
        for i in range(len(arr)):
            x = arr[i][0][0]
            y = arr[i][0][1]
            val = arr[i][1]
            mat[x,y] = val
        population[j] = mat
    return population

In [194]:
def fitness(m):
    side = m.shape[0]
    def repeated(st):
        st, c = np.unique(st, return_counts = True)
        if st[0] != 0: return np.count_nonzero(c > 1)
        return np.count_nonzero(c[1:] > 1)
    result = side**2
    for i in range(side):
        result -= repeated(m[i, :])
        result -= repeated(m[:, i])
        sqrt = np.sqrt(side)
        h, v = (i % sqrt)*sqrt, (i // sqrt)*sqrt
        result -= repeated(m[int(v): int(v + sqrt), int(h): int(h + sqrt)])
    return result - np.count_nonzero(m == 0)

In [195]:
def tournament(fitnesses, k = 2, reversed = False):
    participants = np.random.choice(len(fitnesses), k, False)
    if reversed: return np.argmin(fitnesses[participants])
    return np.argmax(fitnesses[participants])

In [196]:
def simplePermutationCrossover(pR, parents):
    if np.random.uniform(0, 1) < pR:
        return crossover(parents)
    return parents[0]

In [197]:
def crossover(parents):
    first, second = parents
    x = np.random.randint(low=0,high=9)
    offspring = np.zeros((9,9))
    offspring = offspring.astype(int)

    x=4 
    # merge matrixes
    for i in range(x+1):
        for j in range(offspring.shape[0]):
                offspring[i][j] = first[i][j]
    for i in range(x+1, offspring.shape[0]):
        for j in range(offspring.shape[1]):
                offspring[i][j] = second[i][j]
    #print(offspring)

    # missing-duplicated
    blocked = []
    for i in range(offspring.shape[0]):
        coords = []
        dupl = {}
        miss = [1,2,3,4,5,6,7,8,9]
        
        for j in range(offspring.shape[1]):
            coords.append([offspring[j,i], [i,j]])
        # print(coords)
        for j in range(len(coords)):
            rep = np.where(np.array(coords == coords[j][0], dtype=object))
            if(rep[0].shape[0] != 1):
                dupl[coords[j][0]] = rep[0]
        for j in range(len(coords)):
            if(coords[j][0] in miss):
                miss.remove(coords[j][0])
        for j in d:
            blocked.append(j[0])
        for j in dupl.values():
            for z in j:
                if([z,i] not in blocked): # si no está en la lista de bloqueados
                    # print([z,i], miss )
                    if(len(miss)):
                        offspring[z,i] = miss[0]
                        miss.pop(0)
    return offspring 

In [198]:
def mutation(pM, m):
    if np.random.uniform(0, 1) < pM:
        sqrt, side = int(np.sqrt(m.shape[0])), m.shape[0]
        for i in range(side):
            _, indices = np.unique(m[i], return_index=True)
            for j in range(side):
                if j not in indices and m[i, j]:
                    m[i, j], m[(i + sqrt) % side, j] = m[(i + sqrt) % side, j], m[i, j]
    return m

In [205]:
def geneticAlgorithm(f, n, g, pR, pM): #File name, Population Size, # of generations, P(reproduction), p(Mutation), # of queens
    m, d = read_csv(f)
    sideSize = m.shape[0]
    population = initial_population(d, n)
    best, fitnesses = -1, np.array([fitness(i) for i in population])
    best = np.argmax(fitnesses)
    newPopulation, newFitnesses = np.zeros((n, sideSize, sideSize)), np.zeros(n)
    for i in range(g):
        if i % 50 == 0: 
            r = population[best].astype(int)
            print(fitness(r))
            draw_sudoku(r)
        if fitnesses[best] == sideSize**2:
            return population[best].astype(int)
        for childIndex in range(n):
            pOne = population[tournament(fitnesses, n // 2)]
            pTwo = population[tournament(fitnesses, n // 2)]
            #pOne = population[roulette(population)]
            #pTwo = population[roulette(population)]
            parents = np.array([pOne, pTwo])
            newChild = simplePermutationCrossover(pR, parents)
            newChild = mutation(pM, newChild)
            newFitnesses[childIndex] = fitness(newChild)
            newPopulation[childIndex] = newChild
        betterBest = np.argmax(newFitnesses)
        print(fitnesses, newFitnesses)
        if newFitnesses[betterBest] < fitnesses[best]: newPopulation[tournament(fitnesses, n//2, True)] = population[best]
        else: best = betterBest
        population, fitnesses = newPopulation, newFitnesses
    return population[best].astype(int)
r = geneticAlgorithm('extraEasy.csv', 30, 100000, 0.75, 0.75)
print(fitness(r))
draw_sudoku(r)

78
+---+---+---+---+---+---+---+---+---+
| 8   2   7 | 1   5   4 | 3   9   6 |
| 9   6   5 | 0   2   7 | 1   4   8 |
| 3   4   1 | 6   8   9 | 7   5   2 |
+---+---+---+---+---+---+---+---+---+
| 5   9   3 | 4   6   8 | 2   7   1 |
| 4   7   2 | 5   1   3 | 6   8   9 |
| 6   1   8 | 9   7   2 | 4   3   5 |
+---+---+---+---+---+---+---+---+---+
| 7   8   6 | 2   3   5 | 9   1   4 |
| 1   5   4 | 7   9   6 | 8   2   3 |
| 2   3   9 | 0   4   1 | 5   6   0 |
+---+---+---+---+---+---+---+---+---+
78
+---+---+---+---+---+---+---+---+---+
| 8   2   7 | 1   5   4 | 3   9   6 |
| 9   6   5 | 2   2   7 | 1   4   8 |
| 3   4   1 | 6   8   9 | 7   5   2 |
+---+---+---+---+---+---+---+---+---+
| 5   9   3 | 4   6   8 | 2   7   1 |
| 4   7   2 | 5   1   3 | 6   8   9 |
| 6   1   8 | 9   7   2 | 4   3   5 |
+---+---+---+---+---+---+---+---+---+
| 7   8   6 | 2   3   5 | 9   1   4 |
| 1   5   4 | 7   9   6 | 8   2   3 |
| 2   3   9 | 8   4   1 | 5   6   7 |
+---+---+---+---+---+---+---+---+---+
[75 72

/tmp/ipykernel_7574/3912692868.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  rep = np.where(np.array(coords == coords[j][0], dtype=object))


[63. 63. 70. 61. 69. 57. 60. 61. 54. 57. 52. 57. 57. 66. 52. 55. 56. 59.
 65. 65. 66. 57. 57. 60. 55. 55. 56. 54. 63. 57.] [63. 63. 70. 61. 69. 57. 60. 61. 54. 57. 52. 57. 57. 66. 52. 55. 56. 59.
 65. 65. 66. 57. 57. 60. 55. 55. 56. 54. 63. 57.]
[67. 69. 52. 51. 57. 61. 60. 56. 54. 47. 59. 54. 57. 54. 52. 56. 56. 50.
 56. 56. 60. 52. 57. 53. 58. 51. 56. 55. 58. 57.] [67. 69. 52. 51. 57. 61. 60. 56. 54. 47. 59. 54. 57. 54. 52. 56. 56. 50.
 56. 56. 60. 52. 57. 53. 58. 51. 56. 55. 58. 57.]
[57. 66. 56. 56. 61. 53. 56. 54. 47. 56. 54. 56. 59. 56. 56. 56. 54. 50.
 59. 46. 58. 56. 52. 53. 53. 58. 57. 60. 57. 57.] [57. 66. 56. 56. 61. 53. 56. 54. 47. 56. 54. 56. 59. 56. 56. 56. 54. 50.
 59. 46. 58. 56. 52. 53. 53. 58. 57. 60. 57. 57.]
[57. 56. 58. 56. 55. 58. 60. 55. 51. 60. 59. 58. 61. 53. 55. 56. 51. 51.
 63. 52. 53. 55. 56. 55. 56. 55. 57. 60. 55. 54.] [57. 56. 58. 56. 55. 58. 60. 55. 51. 60. 59. 58. 61. 53. 55. 56. 51. 51.
 63. 52. 53. 55. 56. 55. 56. 55. 57. 60. 55. 54.]
[54. 55. 57. 62.

KeyboardInterrupt: 